In [14]:
import logging, sys, os
import numpy as np
import pandas as pd
import sqlite3

from pipeline import construct_twitter_pipeline

In [40]:
tweets_loader = construct_twitter_pipeline().extractor.file_handlers[0].loader
try:
    db_path = os.path.join(tweets_loader.output_directory, tweets_loader.db_name)
    print(db_path)
    connection = sqlite3.connect("../data/project.sqlite")
    tweets = pd.read_sql_query(f"SELECT * FROM {tweets_loader.table_name}", connection)
except sqlite3.Error as e:
    logging.error(msg=f"Error while creating SQLite DB: {e}")
    sys.exit(1)
finally:
    connection.close()

/Users/pranto/Documents/Codes/project-made/made-template/project/data/project.sqlite


In [49]:
# ax = tweets.plot.bar(x='category', y=tweets['category'].value_counts(), rot=0)
tweets.groupby('category').count()

,tweets
category,
-1.0,35509
0.0,55211
1.0,72249
